In [2]:
pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 27.9 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.3.7
    Uninstalling Werkzeug-2.3.7:
      Successfully uninstalled Werkzeug-2.3.7


In [3]:
from collections import defaultdict, Counter
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import json
import numpy as np

<ipython-input-3-48586731e8ae>:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-3-48586731e8ae>:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


Open the data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#Edit your path
%cd '/content/drive/MyDrive/softpr_sose23/Datasets/characters_disorders'

/content/drive/.shortcut-targets-by-id/1s-EhtApF2sIWDdSVA4ieNTqZygjiySaa/softpr_sose23/Datasets/characters_disorders


## Creating a database of characters by country/author/gender

In [6]:
data = defaultdict()
countries = ['russia', 'france', 'germany', 'england']
countries_chars_dfs = defaultdict()

In [ ]:
authors = dict()
authors['russia'] = ['Dostoyevsky', 'Turgenev', 'Tolstoy']
authors['france'] = ['Hugo', 'Dumas', 'Sand']
authors['germany'] = ['Goethe', 'Fontane', "Hoffmann"]
authors['england'] = ['Austen', 'Wilde', 'Woolf']

In [ ]:
data['country'] = {}
for country in countries:
  data['country'][country] = {}
  for author in authors[country]:
    data['country'][country][author] = {}

In [ ]:
for country in countries:
  countries_chars_dfs[country] = {}

  for author in authors[country]:
    countries_chars_dfs[country][author] = pd.read_csv(f'{author.lower()}_disorders.csv')

In [ ]:
#Turn strings into lists
for country in countries_chars_dfs:
  for author in countries_chars_dfs[country]:
    df_chars = countries_chars_dfs[country][author]
    for i in range(len(df_chars)):
      df_chars['similarity_scores'][i] = eval(df_chars['similarity_scores'][i])

<ipython-input-80-29091b222562>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chars['similarity_scores'][i] = eval(df_chars['similarity_scores'][i])


In [ ]:
males = defaultdict()
females = defaultdict()

for country in countries_chars_dfs:
  males[country] = {}
  females[country] = {}
  for author in authors[country]:
    df_chars = countries_chars_dfs[country][author]
    males[country][author] = df_chars[df_chars['gender'] == 'm']
    females[country][author] = df_chars[df_chars['gender'] == 'f']


In [ ]:
male_disorders = defaultdict()
female_disorders = defaultdict()
#by country and then author

In [ ]:
for country in countries_chars_dfs:
  male_disorders[country] = {}
  female_disorders[country] = {}
  for author in authors[country]:
    male_disorders[country][author] = dict(Counter([el[0][0] for el in males[country][author]['similarity_scores']]).most_common())
    female_disorders[country][author] = dict(Counter([el[0][0] for el in females[country][author]['similarity_scores']]).most_common())s

In [ ]:
united_disorders = defaultdict()
for country in countries_chars_dfs:
  united_disorders[country] = {}
  for author in authors[country]:
    united_disorders[country][author] = []
    united_disorders[country][author].extend(list(male_disorders[country][author].keys()))
    united_disorders[country][author].extend(list(female_disorders[country][author].keys()))
    united_disorders[country][author] = set(united_disorders[country][author])



In [ ]:
for country in countries_chars_dfs:
  for author in authors[country]:
    for dis in united_disorders[country][author]:
      data['country'][country][author][dis] = {'male': 0, 'female': 0}
    for dis in male_disorders[country][author]:
      try:
        data['country'][country][author][dis]['male'] = male_disorders[country][author][dis]
      except:
        next
    for dis in female_disorders[country][author]:
      try:
        data['country'][country][author][dis]['female'] = female_disorders[country][author][dis]
      except:
        next


In [ ]:
#save the file
json_file_path = '/content/drive/MyDrive/softpr_sose23/Datasets/characters_disorders/vis_data.json'

with open(json_file_path, 'w') as json_file:
    json.dump(data, json_file)

Visualize without dash

In [ ]:
json_file_path = '/content/drive/MyDrive/softpr_sose23/Datasets/characters_disorders/vis_data.json'
with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)

## Visualize with dash

In [ ]:
json_file_path = '/content/drive/MyDrive/softpr_sose23/Datasets/characters_disorders/vis_data.json'
with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)

df = pd.DataFrame(data)

# Initialize Dash app
app = dash.Dash(__name__)
server = app.server

# Layout of the app
app.layout = html.Div([
    html.H1('Distribution of Mental Disorders'),
    html.Div([
        html.Label('Select a Country:'),
        dcc.Dropdown(
            id='country-dropdown',
            options=[{'label': country.capitalize(), 'value': country} for country in list(data['country'].keys())],
            value=list(data['country'].keys())[0],
            multi=False
        ),
        html.Label('Select an Author:'),
        dcc.Dropdown(id='author-dropdown')
    ]),
    dcc.Graph(id='disorders-by-gender-and-country'),
    dcc.Graph(id='disorders-by-gender-and-author'),
    # Placeholder div for the interactive plot
    html.Div(id='interactive-plot-container')
])

# Callback to populate author dropdown based on selected country
@app.callback(
    Output('author-dropdown', 'options'),
    [Input('country-dropdown', 'value')]
)
def update_author_dropdown(selected_country):
    authors = df['country'][selected_country].keys()
    options = [{'label': author, 'value': author} for author in authors]
    return options

# Callback to update the first bar chart (disorders by gender and country)
@app.callback(
    Output('disorders-by-gender-and-country', 'figure'),
    [Input('country-dropdown', 'value')]
)
def update_bar_chart_country(selected_country):
    # Create separate DataFrames for males and females
  filtered_df = pd.DataFrame(df['country'][selected_country])

  # Reset the index and move it into a new column
  filtered_df.reset_index(inplace=True)

  # Rename the new column (optional)
  filtered_df.rename(columns={'index': 'disorders'}, inplace=True)

  male, female = [], []
  for ind in filtered_df.index:
    male_interm = 0
    female_interm = 0
    for author in filtered_df.keys():
      try:
        male_interm += filtered_df[author][ind]['male']
        female_interm += filtered_df[author][ind]['female']
      except:
        next
    male.append(male_interm)
    female.append(female_interm)

  filtered_df['male'] = male
  filtered_df['female'] = female
  authors = df['country'][selected_country].keys()

  filtered_df = filtered_df.drop(columns=[col for col in authors])
  melted_df = filtered_df.melt(id_vars=['disorders'], var_name='Gender', value_name='Count')
  male_df = melted_df[melted_df['Gender'] == 'male']
  female_df = melted_df[melted_df['Gender'] == 'female']

  # Define distinct color sequences for males and females
  color_sequence_male = ['#1f77b4', '#aec7e8']  # Blue shades
  color_sequence_female = ['#ff7f0e', '#ffbb78']  # Orange shades

  # Create individual bar charts for males and females with distinct colors
  fig = px.bar(male_df, x='Count', y='disorders', orientation='h', text='Count',
              title=f'Distribution of Disorders by Gender in {selected_country}',
              color='Gender', color_discrete_sequence=color_sequence_male,
              labels={'Gender': 'Legend'})
  fig.update_traces(texttemplate='%{text}', textposition='inside', marker=dict(line=dict(color='navy', width=1)))

  fig2 = px.bar(female_df, x='Count', y='disorders', orientation='h', text='Count',
                title=f'Distribution of Disorders by Gender in {selected_country.capitalize()}',
                color='Gender', color_discrete_sequence=color_sequence_female,
                labels={'Gender': 'Legend'})
  fig2.update_traces(texttemplate='%{text}', textposition='inside', marker=dict(line=dict(color='firebrick', width=1)))

  # Combine the two figures
  fig.add_trace(fig2.data[0])

  # Customize layout
  fig.update_layout(barmode='relative', xaxis_tickangle=-45,
                    xaxis_title='Total count of Cases', yaxis_title='Disorders',
                    paper_bgcolor='white')

  # Adjust the figure size
  fig.update_layout(width=1000, height=600)

  # Show the population pyramid plot
  return fig

# Callback to update the second bar chart (disorders by gender and author within selected country)
@app.callback(
    Output('disorders-by-gender-and-author', 'figure'),
    [Input('country-dropdown', 'value'),
     Input('author-dropdown', 'value')]
)
def update_bar_chart_author(selected_country, selected_author):
    filtered_df = pd.DataFrame(df['country'][selected_country])
    filtered_df.reset_index(inplace=True)
    # Rename the new column (optional)
    filtered_df.rename(columns={'index': 'disorders'}, inplace=True)
    melted_df = filtered_df.melt(id_vars=['disorders'], var_name='author')

    male, female = [], []
    for ind in range(len(melted_df)):
      try:
        male.append(melted_df['value'][ind]['male'])
        female.append(melted_df['value'][ind]['female'])
      except:
        male.append(0)
        female.append(0)

    melted_df['male'] = male
    melted_df['female'] = female
    #delete all rows where the values are both 0
    melted_df = melted_df[(melted_df['male'] != 0) | (melted_df['female'] != 0)]
    melted_df = melted_df.drop(columns = ['value'])
    melted_df = melted_df.melt(id_vars=['author', 'disorders'], var_name='gender', value_name='count')
    melted_df = melted_df[(melted_df['author'] == selected_author)]
    fig = px.bar(melted_df, x='disorders', y='count', color='gender', barmode='group',
                title=f'Distribution of Disorders by Gender and Author in {selected_country.capitalize()} - {selected_author}')


    return fig


# Run the app
if __name__ == '__main__':

    app.run_server(debug=True)

<IPython.core.display.Javascript object>